# Análise Exploratória

In [1]:
# Importando as Bibliotecas
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as f

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Inicialização Spark e carregamento dos dados

In [2]:
spark = SparkSession.builder \
    .master('local[*]') \
    .appName('Forecasting com Spark') \
    .getOrCreate()

spark

In [3]:
# Antes de carregar os dados, vamos explicitar a estrutura dos dados que
# serão carregados

schema = StructType([
    StructField('day_of_purchase', StringType(), False), # será carregado como string para posterior manipulação
    StructField('seller_id', StringType(), False),
    StructField('customer_id', StringType(), False),
    StructField('order_id', StringType(), False),
    StructField('order_status', StringType(), False),
    StructField('order_item_id', IntegerType(), False),
    StructField('product_id', StringType(), False),
    StructField('product_category_name', StringType(), False),
    StructField('price', DoubleType(), False),
    StructField('freight_value', DoubleType(), False),
    StructField('customer_city', StringType(), False),
    StructField('seller_city', StringType(), False)
])

In [4]:
#Importando os dados
path_csv = '../dados/forecast_query.csv'

df = spark.read.csv(
    path_csv,
    inferSchema=True,
    header=True,
    schema=schema
)

## Compreensão dos Dados

In [6]:
# Verificando se os tipos foram determinados corretamente de acordo com o 
# struct type previamente definido
df.printSchema()

root
 |-- day_of_purchase: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_category_name: string (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- seller_city: string (nullable = true)



In [5]:
# Visualizar as primeiras cinco linhas do Spark Dataframe
df.limit(5).toPandas()

,day_of_purchase,seller_id,customer_id,order_id,order_status,order_item_id,product_id,product_category_name,price,freight_value,customer_city,seller_city
0,2016-09-04 21:15:19,1554a68530182680ad5c8b042c3ab563,08c5351a6aca1c1589a38f244edeee9d,2e7a8482f6fb09756ca50c10d7bfc047,shipped,1,c1488892604e4ba5cff5b4eb4d595400,moveis_decoracao,39.99,31.67,boa vista,monte siao
1,2016-09-04 21:15:19,1554a68530182680ad5c8b042c3ab563,08c5351a6aca1c1589a38f244edeee9d,2e7a8482f6fb09756ca50c10d7bfc047,shipped,2,f293394c72c9b5fafd7023301fc21fc2,moveis_decoracao,32.90,31.67,boa vista,monte siao
2,2016-09-15 12:16:38,ecccfa2bb93b34a3bf033cc5d1dcdc69,86dc2ffce2dfff336de2f386a786e574,bfbd0f9bdef84302105ad712db648a6c,delivered,1,5a6b04657a4c5ee34285d1e4619a96b4,beleza_saude,44.99,2.83,sao joaquim da barra,curitiba
3,2016-09-15 12:16:38,ecccfa2bb93b34a3bf033cc5d1dcdc69,86dc2ffce2dfff336de2f386a786e574,bfbd0f9bdef84302105ad712db648a6c,delivered,2,5a6b04657a4c5ee34285d1e4619a96b4,beleza_saude,44.99,2.83,sao joaquim da barra,curitiba
4,2016-09-15 12:16:38,ecccfa2bb93b34a3bf033cc5d1dcdc69,86dc2ffce2dfff336de2f386a786e574,bfbd0f9bdef84302105ad712db648a6c,delivered,3,5a6b04657a4c5ee34285d1e4619a96b4,beleza_saude,44.99,2.83,sao joaquim da barra,curitiba


In [7]:
# Verificando quantos registros temos no dataset
print('Quantidade de vendas:', df.count())

Quantidade de vendas: 112108


In [8]:
# Verificando se existen colunas com valores nulos
df.select(
    [f.count(f.when(f.isnan(c) | f.col(c).isNull(), c)).alias(c) 
        for c in df.columns]

).toPandas()

,day_of_purchase,seller_id,customer_id,order_id,order_status,order_item_id,product_id,product_category_name,price,freight_value,customer_city,seller_city
0,0,0,0,0,0,0,0,0,0,0,0,0


In [9]:
# Verificando qtas categorias de produto o dataset possui
df\
    .select(f.count_distinct('product_category_name').alias('qtd categorias'))\
    .show()


+--------------+
|qtd categorias|
+--------------+
|            74|
+--------------+



Existem várias categorias neste dataset, e talvez não seja viável trabalharmos com todos eles.  
Vamos investigar melhor as categorias para vermos se podemos refinar as pesquisas.

In [57]:
# Ordenando por quantidade de produtos
df\
    .select('price', 'product_category_name')\
    .groupBy('product_category_name')\
    .agg(
        f.round(f.sum('price'), 2).alias('total_preco'),
        f.count('product_category_name').alias('qtd_produtos')
    )\
    .orderBy('qtd_produtos', ascending=False)\
    .limit(20)\
    .toPandas()

,product_category_name,total_preco,qtd_produtos
0,cama_mesa_banho,1035964.06,11097
1,beleza_saude,1255695.13,9634
2,esporte_lazer,979740.92,8590
3,moveis_decoracao,727465.05,8298
4,informatica_acessorios,904322.02,7781
5,utilidades_domesticas,626825.80,6915
6,relogios_presentes,1198185.21,5970
7,telefonia,322342.64,4527
8,ferramentas_jardim,481009.94,4328
9,automotivo,586620.73,4205


In [55]:
# Ordenando pelo preco do produto
df\
    .select('price', 'product_category_name')\
    .groupBy('product_category_name')\
    .agg(
        f.round(f.sum('price'), 2).alias('total_preco'),
        f.count('product_category_name').alias('qtd_produtos')
    )\
    .orderBy('total_preco', ascending=False)\
    .limit(20)\
    .toPandas()
    # .show(truncate=False)
    

,product_category_name,total_preco,qtd_produtos
0,beleza_saude,1255695.13,9634
1,relogios_presentes,1198185.21,5970
2,cama_mesa_banho,1035964.06,11097
3,esporte_lazer,979740.92,8590
4,informatica_acessorios,904322.02,7781
5,moveis_decoracao,727465.05,8298
6,utilidades_domesticas,626825.80,6915
7,cool_stuff,620835.39,3780
8,automotivo,586620.73,4205
9,ferramentas_jardim,481009.94,4328


Ao ordenar pela quantidade, notamos que existe uma categoria de produto que está como NULL.  
Provavelmente saiu assim quando realizamos a query no banco. Vamos deletar esses itens futuramente.

In [12]:
# Transformação do 'Valor total'
# df.withColumn(
#     'total_value',
#     df['price'] + df['freight_value']
# ).limit(5).toPandas()

,day_of_purchase,seller_id,customer_id,order_id,order_status,order_item_id,product_id,product_category_name,price,freight_value,customer_city,seller_city,total_value
0,2016-09-04 21:15:19,1554a68530182680ad5c8b042c3ab563,08c5351a6aca1c1589a38f244edeee9d,2e7a8482f6fb09756ca50c10d7bfc047,shipped,1,c1488892604e4ba5cff5b4eb4d595400,moveis_decoracao,39.99,31.67,boa vista,monte siao,71.66
1,2016-09-04 21:15:19,1554a68530182680ad5c8b042c3ab563,08c5351a6aca1c1589a38f244edeee9d,2e7a8482f6fb09756ca50c10d7bfc047,shipped,2,f293394c72c9b5fafd7023301fc21fc2,moveis_decoracao,32.90,31.67,boa vista,monte siao,64.57
2,2016-09-15 12:16:38,ecccfa2bb93b34a3bf033cc5d1dcdc69,86dc2ffce2dfff336de2f386a786e574,bfbd0f9bdef84302105ad712db648a6c,delivered,1,5a6b04657a4c5ee34285d1e4619a96b4,beleza_saude,44.99,2.83,sao joaquim da barra,curitiba,47.82
3,2016-09-15 12:16:38,ecccfa2bb93b34a3bf033cc5d1dcdc69,86dc2ffce2dfff336de2f386a786e574,bfbd0f9bdef84302105ad712db648a6c,delivered,2,5a6b04657a4c5ee34285d1e4619a96b4,beleza_saude,44.99,2.83,sao joaquim da barra,curitiba,47.82
4,2016-09-15 12:16:38,ecccfa2bb93b34a3bf033cc5d1dcdc69,86dc2ffce2dfff336de2f386a786e574,bfbd0f9bdef84302105ad712db648a6c,delivered,3,5a6b04657a4c5ee34285d1e4619a96b4,beleza_saude,44.99,2.83,sao joaquim da barra,curitiba,47.82
